In [5]:
import aiomysql
import pandas as pd

In [3]:
!pip install aiomysql

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for aiomysql from https://files.pythonhosted.org/packages/42/87/c982ee8b333c85b8ae16306387d703a1fcdfc81a2f3f15a24820ab1a512d/aiomysql-0.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for PyMySQL>=1.0 from https://files.pythonhosted.org/packages/e5/30/20467e39523d0cfc2b6227902d3687a16364307260c75e6a1cb4422b0c62/PyMySQL-1.1.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 588.0 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.5 MB/s eta 0:00:00


In [5]:
async def collect_candle(limit=None):
    loop = asyncio.get_event_loop()
    pool = await aiomysql.create_pool(host=config['SQL']['host'],
                                      port=3306,
                                      user = config['SQL']['user'],
                                      password = config['SQL']['password'],
                                      db=config['SQL']['db_name'],
                                      loop=loop)
    async with pool.acquire() as con:
        cur = await con.cursor()
        await cur.execute('SELECT * FROM AKRN')
        if limit:
            rows = cur.fetchmany(limit)
        else:
            rows = cur.fetchall()
        print(rows)
    return rows

In [1]:
import mysql.connector
# Параметры хоста
host = '91.106.207.34'
user = 'asgitma8_exchang' # для базы минутных свечей за 190 дней 'asgitma8_exch_1'
password = 'DataBase2023'
db_name = 'asgitma8_exchang' # для базы минутных свечей за 190 дней 'asgitma8_exch_1'

In [2]:
# Функция получения свечей из базы данных
def get_candles(host, user, password, db_name):
    query = 'SELECT * FROM operative_candles'
    # Создаём подключение 
    myconn = mysql.connector.connect(host=host, user=user, passwd=password,
                                     database=db_name)
    # Создаём курсора 
    cur = myconn.cursor() 
    try: 
        # Выполняем запрос к базе данных 
        cur.execute(query)
        # Выгружаем результаты запроса 
        candles = cur.fetchall() 
    except:
        # Откатываем изменения в случае ошибки соединения с базой
        myconn.rollback()
        print('Ошибка при работе с базой!')
    # Закрываем соединение с базой
    myconn.close()
    return candles

In [48]:
query = """
    SELECT oc.candle_time as 'ВРЕМЯ СВЕЧИ',
           s.ticker as 'ТИКЕР',
           s.figi as 'ИДЕНТИФИКАТОР',
           oc.volume as 'ОБЪЁМ ПРОДАЖ',
           oc.is_complete as 'БЫЛА ИНФОРМАЦИЯ',
           oc.now_time as 'ВРЕМЯ ПОСТУПЛЕНИЯ СВЕЧИ',
           microsecond(oc.now_time)/1000000  as 'ВРЕМЯ ОТВЕТА СЕРВЕРА, сек'
    FROM operative_candles oc
    LEFT JOIN shares s
           ON s.figi = oc.figi
"""
myconn = mysql.connector.connect(host=host, user=user, passwd=password,
                                     database=db_name)

In [49]:
df = pd.read_sql(query, myconn)

/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [50]:
df

,ВРЕМЯ СВЕЧИ,ТИКЕР,ИДЕНТИФИКАТОР,ОБЪЁМ ПРОДАЖ,БЫЛА ИНФОРМАЦИЯ,ВРЕМЯ ПОСТУПЛЕНИЯ СВЕЧИ,"ВРЕМЯ ОТВЕТА СЕРВЕРА, сек"
0,2023-10-06 13:38:00,ENPG,BBG000RMWQD4,14,True,2023-10-06 13:39:57.400101,0.4001
1,1970-01-01 00:00:00,ENPG,BBG000RMWQD4,0,False,2023-10-06 13:40:57.078316,0.0783
2,2023-10-06 13:40:00,ENPG,BBG000RMWQD4,100,True,2023-10-06 13:41:57.045936,0.0459
3,2023-10-06 13:40:00,ENPG,BBG000RMWQD4,100,True,2023-10-06 13:42:57.038146,0.0381
4,2023-10-06 13:40:00,ENPG,BBG000RMWQD4,100,True,2023-10-06 13:43:57.067787,0.0678
...,...,...,...,...,...,...,...
1492,2023-10-06 14:08:00,None,None,10,True,2023-10-06 14:09:57.038808,0.0388
1493,2023-10-06 14:09:00,None,None,14,True,2023-10-06 14:10:57.045696,0.0457
1494,2023-10-06 14:10:00,None,None,7,True,2023-10-06 14:11:57.041985,0.0420
1495,2023-10-06 14:11:00,None,None,12,True,2023-10-06 14:12:57.049976,0.0500


In [51]:
df.to_excel('Оперативные свечи за полчаса (для проверки релевантности)_ticker.xlsx', index=False)

In [47]:
df.nunique()

ВРЕМЯ СВЕЧИ                  36
ТИКЕР                        60
ИДЕНТИФИКАТОР                60
ОБЪЁМ ПРОДАЖ                574
БЫЛА ИНФОРМАЦИЯ               2
ВРЕМЯ ПОСТУПЛЕНИЯ СВЕЧИ    1497
ВРЕМЯ ОТВЕТА СЕРВЕРА        669
dtype: int64

In [54]:
sum([1, 4])

5